In [2]:
# tools/vm_cpu_tool.py
from langchain.tools import tool
import requests
import statistics
import datetime

VM_URL = "http://localhost:8428/prometheus/api/v1"

@tool
def query_cpu(host: str, start: str, end: str, step: str = "5m") -> dict:
    """
    Query CPU usage for a given host from VictoriaMetrics and return statistics.
    
    Args:
        host: Hostname or ID.
        start: Start time (RFC3339).
        end: End time (RFC3339).
        step: Resolution step (default: 5m).
    
    Returns:
        Dict with CPU statistics and sample points.
    """
    query = f"nab_aws_cpu_value{{host='{host}'}}"
    url = f"{VM_URL}/query_range"
    params = {"query": query, "start": start, "end": end, "step": step}

    resp = requests.get(url, params=params, timeout=60)
    if resp.status_code != 200:
        return {"error": f"Query failed: {resp.text}"}

    data = resp.json().get("data", {}).get("result", [])
    if not data:
        return {"error": "No data found for given host/time range"}

    # Flatten values into list
    values = []
    timestamps = []
    for series in data:
        for ts, val in series["values"]:
            try:
                v = float(val)
                values.append(v)
                timestamps.append(datetime.datetime.utcfromtimestamp(float(ts)).isoformat())
            except:
                continue

    if not values:
        return {"error": "No valid numeric values found"}

    # Compute stats
    stats = {
        "count": len(values),
        "min": min(values),
        "max": max(values),
        "mean": statistics.mean(values),
        "median": statistics.median(values),
        "stdev": statistics.pstdev(values) if len(values) > 1 else 0,
        "first_point": {"time": timestamps[0], "value": values[0]},
        "last_point": {"time": timestamps[-1], "value": values[-1]},
    }

    return {
        "host": host,
        "start": start,
        "end": end,
        "step": step,
        "stats": stats,
        "sample_values": values[:10],  # only first 10 values to avoid overload
    }